<a href="https://colab.research.google.com/gist/MBcode/8680047033fccd284744fd60f47b504d/ingesttesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GeoCODES ingest pipeline testing:

#Load the github.com/earthcube/earthcube_utilities 
 for now the staging version

In [ ]:
%load_ext rpy2.ipython
%load_ext google.colab.data_table
import httpimport
with httpimport.github_repo('MBcode', 'ec'):
  import ec

# Start by seeing how many records are expected from each repository
To be harvested by google-datasetsearch or GeoCODES only happens if the repository has a 'sitemap.xml' listing the dataset landing pages, which have jsonld in them, that they want indexed, so the search engine can find dataset resources of interest So we start by crawling the sitemaps, (but just this time) we will only record the number of records, so we can know how many to expect from each repository, during in the next parts of the ingestion pipeline

In [ ]:
sitemaps=[
"https://www.unavco.org/data/doi/sitemap.xml",
"https://ssdb.iodp.org/dataset/sitemap.xml",
#"http://balto.opendap.org/opendap/site_map.txt ",
"http://wiki.linked.earth/sitemap.xml",
"https://lipdverse.org/sitemap.xml",
"http://ds.iris.edu/files/sitemap.xml",
"https://data.ucar.edu/sitemap.xml",
"http://opencoredata.org/sitemap.xml",
"https://www2.earthref.org/MagIC/contributions.sitemap.xml",
#"http://data.neotomadb.org/sitemap.xml",
"https://ecl.earthchem.org/sitemap.xml",
"https://xdomes.tamucc.edu/srr/sensorML/sitemap.xml",
"https://geodex.org/neon_prodcodes_sm.xml",
#"https://www.designsafe-ci.org/sitemap.xml ",
"https://object.cloud.sdsc.edu/v1/AUTH_85f46aa78936477d8e71b186269414e8/gleaner-summoned",
"http://datadiscoverystudio.org/sitemap/CinergiSiteIndex.xml",
"https://www.unidata.ucar.edu/sitemap.xml",
"https://service-dev.rvdata.us/api/sitemap/",
"https://raw.githubusercontent.com/earthcube/GeoCODES-Metadata/gh-pages/metadata/Dataset/sitemap.xml",
"https://www.usap-dc.org/view/dataset/sitemap.xml",
"https://cchdo.ucsd.edu/sitemap.xml",
"https://amgeo-dev.colorado.edu/sitemap.xml "]

In [ ]:
ec.setup_sitemap()

In [ ]:
sitemaps_count = ec.sitemaps_count(sitemaps)

In [ ]:
sitemap_count

{'http://datadiscoverystudio.org/sitemap/CinergiSiteIndex.xml': 0,
 'http://ds.iris.edu/files/sitemap.xml': 0,
 'http://opencoredata.org/sitemap.xml': 83388,
 'http://wiki.linked.earth/sitemap.xml': 18634,
 'https://cchdo.ucsd.edu/sitemap.xml': 5040,
 'https://data.ucar.edu/sitemap.xml': 17005,
 'https://ecl.earthchem.org/sitemap.xml': 920,
 'https://geodex.org/neon_prodcodes_sm.xml': 0,
 'https://lipdverse.org/sitemap.xml': 704,
 'https://object.cloud.sdsc.edu/v1/AUTH_85f46aa78936477d8e71b186269414e8/gleaner-summoned': 0,
 'https://raw.githubusercontent.com/earthcube/GeoCODES-Metadata/gh-pages/metadata/Dataset/sitemap.xml': 0,
 'https://service-dev.rvdata.us/api/sitemap/': 0,
 'https://ssdb.iodp.org/dataset/sitemap.xml': 25241,
 'https://www.unavco.org/data/doi/sitemap.xml': 0,
 'https://www.unidata.ucar.edu/sitemap.xml': 211,
 'https://www.usap-dc.org/view/dataset/sitemap.xml': 0,
 'https://www2.earthref.org/MagIC/contributions.sitemap.xml': 0,
 'https://xdomes.tamucc.edu/srr/sensorM

# Next see how many made it into our LinkedData-cache
Once we capture the jsonld from each landing page, we cache that LinkedData as is, and then in another RDF format: .nt ntriples, that is easier to concatenate and to load into a triplestore

In [ ]:
ec.setup_s3fs()

In [ ]:
import s3fs

In [ ]:
oss = s3fs.S3FileSystem(
      anon=True,
      key="",
      secret="",
      client_kwargs = {"endpoint_url":"https://oss.geodex.org"}
   )

In [ ]:
summoned =oss.ls('gleaner/summoned')
milled =oss.ls('gleaner/milled')

In [ ]:
for repo in summoned:
  fa=oss.ls(repo)
  fnum=len(fa)
  print(f'repo:{repo} has {fnum} files')

repo:gleaner/summoned/amgeo has 5 files
repo:gleaner/summoned/aquadocs has 1 files
repo:gleaner/summoned/bco-dmo has 13221 files
repo:gleaner/summoned/bcodmo has 12484 files
repo:gleaner/summoned/cchdo has 4235 files
repo:gleaner/summoned/designsafe has 352 files
repo:gleaner/summoned/earthchem has 1549 files
repo:gleaner/summoned/edi has 14338 files
repo:gleaner/summoned/geocodes_demo_datasets has 8 files
repo:gleaner/summoned/getiedadataorg has 7206 files
repo:gleaner/summoned/hydroshare has 15297 files
repo:gleaner/summoned/ieda has 7774 files
repo:gleaner/summoned/iedadata has 9347 files
repo:gleaner/summoned/iris has 63 files
repo:gleaner/summoned/linked.earth has 696 files
repo:gleaner/summoned/lipdverse has 698 files
repo:gleaner/summoned/magic has 10725 files
repo:gleaner/summoned/neon has 181 files
repo:gleaner/summoned/ocd has 104951 files
repo:gleaner/summoned/opencoredata has 5894 files
repo:gleaner/summoned/opentopo has 666 files
repo:gleaner/summoned/opentopography has 70

# Finally see how many get into the triplestore that backs 

Each dataset landing page's embeded ld+json jsonld record is put into it's own graph withen the store The graph is labeled with a UniversalResourceName [URN](https://en.wikipedia.org/wiki/Uniform_Resource_Name) which is made with a template of type urn:{type}:{repo}:... So we collect all the URNs, and count how many there are for each repository If we are very lucky, the sitemap count = LD-cache count = this count, but loosing a few from errors does happen

In [ ]:
ec.init_sparql()
df=ec.get_graphs()

In [ ]:
df

,g
0,urn:gleaner2:milled:opentopography:0047cda277b...
1,urn:gleaner2:milled:opentopography:00845ce0c33...
2,urn:gleaner2:milled:opentopography:008970cbb2d...
3,urn:gleaner2:milled:opentopography:00b839b8c22...
4,urn:gleaner2:milled:opentopography:01138abe87a...
...,...
290420,urn:gleaner:prov:r2r:ff91af0a990b49c6fa7cd71f1...
290421,urn:gleaner:prov:r2r:ffc3cd385505936a2df73626c...
290422,urn:gleaner:prov:r2r:ffc9b039d7a4c355a8cadf681...
290423,urn:gleaner:prov:r2r:ffdacfad983e5c8d8b0be4e87...


In [ ]:
#will do in pandas, but 1st pass was w/csv dump, like:
df.to_csv("graphs.csv")

In [ ]:
repoCounts=ec.os_system_("cut -d':' -f3,4 graphs.csv | grep milled | sort | uniq -c |sort -n")
repoCounts

'      3 milled:geocodes_demo_datasets\n      5 milled:amgeo\n     83 milled:iris\n    182 milled:neon\n    697 milled:lipidverse\n    842 milled:usap-dc\n    892 milled:designsafe\n   1379 milled:linked.earth\n   1649 milled:r2r\n   1806 milled:cchdo\n   1823 milled:earthchem\n   2615 milled:opentopography\n   5646 milled:hydroshare\n   7807 milled:edi\n  10793 milled:iedadata\n  12743 milled:magic\n  12902 milled:bco-dmo\n  13379 milled:ucar\n'

In [ ]:
!cut -d':' -f3,4 graphs.csv | grep milled | sort | uniq -c |sort -n 

      3 milled:geocodes_demo_datasets
      5 milled:amgeo
     83 milled:iris
    182 milled:neon
    697 milled:lipidverse
    842 milled:usap-dc
    892 milled:designsafe
   1379 milled:linked.earth
   1649 milled:r2r
   1806 milled:cchdo
   1823 milled:earthchem
   2615 milled:opentopography
   5646 milled:hydroshare
   7807 milled:edi
  10793 milled:iedadata
  12743 milled:magic
  12902 milled:bco-dmo
  13379 milled:ucar


put in org info in store, so can query to get a table of the falloff between stages